In [34]:
import mysql.connector as SQLC
class DB:
    
    def __init__(self):
        self.cnx = SQLC.connect(
                host ="geo.di.uminho.pt",
                user ="bioinformatica",
                password ="20221207",
                database ="AP_db_KRG")
        self.sequences = []
        
    def extract_sequences(self):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene")
        self.sequences = cursor.fetchall()
        cursor.close()
    
    def __str__(self): 
        return self.sequences

In [2]:
class Blast:
    def __init__(self, w, seq):
        self.w = w
        self.seq = seq
        
    def query_map(self, query):
    #returns a dictionary where key is query of length w and values are 
    #the indexes of query that match with key
        
        query_map = {}
        for i in range(len(query) - self.w + 1):
            window = query[i: i + self.w]
            if window in query_map:
                query_map[window].append(i)
            else:
                query_map[window] = [i]
        return query_map 

    def hits(self, query_map, sequence):
        #returns list of tuples where 1st index is from query and 2nd is from seq
        hits = []
        for window_query, query_offsets in query_map.items():
            if window_query in sequence:
                seq_offset = sequence.find(window_query)
                while seq_offset != -1: #because of find()
                    for query_offset in query_offsets:
                        hits.append((query_offset, seq_offset))
                    seq_offset = sequence.find(window_query, seq_offset + 1) #while loop and find()
        return hits

    def extend_hit_dir(self, query, sequence, q_offset, s_offset, direction):
        count = 0
        matches = 0
        while q_offset >= 0 and s_offset >= 0 and q_offset < len(query) and s_offset < len(sequence):
            count += 1
            if query[q_offset] == sequence[s_offset]:
                matches += 1
            if 2 * matches < count:
                break
            q_offset += direction
            s_offset += direction
        return q_offset - direction, s_offset - direction, matches, count

    def best_hit(self, query, sequence, w):
        query_map = self.query_map(query)
        hits = self.hits(query_map, sequence)
        best_score = -1
        best_hit = None
        for hit in hits:
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, hit[0], hit[1], 1)
            o1_start, o2_start = o1 - count + 1, o2 - count + 1
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, o1_start, o2_start, -1)
            score = matches / count
            if score > best_score or (score == best_score and count < best_hit[2]):
                best_score = score
                best_hit = (o1_start, o2_start, count, matches)
        return best_hit

In [2]:
#FINAL
class SimpleBlast:
    def __init__(self, w, seq):
        self.w = w
        self.seq = seq
        
    def query_map(self, query):
    #returns a dictionary where key is query of length w and values are 
    #the indexes of query that match with key
        
        query_map = {}
        for i in range(len(query) - self.w + 1):
            window = query[i: i + self.w]
            if window in query_map:
                query_map[window].append(i)
            else:
                query_map[window] = [i]
        return query_map 

    def hits(self, query_map, sequence):
        #returns list of tuples where 1st index is from query and 2nd is from seq
        hits = []
        for window_query, query_offsets in query_map.items():
            if window_query in sequence:
                seq_offset = sequence.find(window_query)
                while seq_offset != -1: #because of find()
                    for query_offset in query_offsets:
                        hits.append((query_offset, seq_offset))
                    seq_offset = sequence.find(window_query, seq_offset + 1) #while loop and find()
        return hits

    def extend_hit_dir(self, query, sequence, q_offset, s_offset, direction):
        #direction: -1 or +1
        count = 0
        matches = 0
        while q_offset >= 0 and s_offset >= 0 and q_offset < len(query) and s_offset < len(sequence):
            count += 1
            if query[q_offset] == sequence[s_offset]:
                matches += 1
            if 2 * matches < count:
                break
            q_offset += direction
            s_offset += direction
        return q_offset - direction, s_offset - direction, matches, count

    def extend_hit(self, query, seq, hit, w):
        
        o1, o2 = hit
        left  = self.extend_hit_dir(query, seq, o1 - 1, o2 - 1, -1)
        right = self.extend_hit_dir(query, seq, o1 + w, o2 + w, +1)

        O1, O2, ML, SL = left
        _,   _, MR, SR = right

#         print({'ML':ML, 'SR':SR, 'SL':SL,'MR':MR})
        return O1, O2, w + SL + SR, ML + w + MR
    
    def best_hit(self, query, seq, w):
        query_map = self.query_map(query)
        hits = self.hits(query_map, seq)
        best_hit = None
        best_score = 0
        for hit in hits:
            o1, o2, l, m = self.extend_hit(query, seq, hit, w)
            score = m - (l - m)
            if score > best_score or (score == best_score and l < best_hit[2]):
                best_hit = (o1, o2, l, m)
                best_score = score
        return best_hit

In [22]:
query = "AATATAT"
seq = "AATATGTTATATAATAATATTT"
w = 3
blast = SimpleBlast(w, seq)
# resulta = blast.query_map(query)
# print(resulta)
# result = blast.hits(blast.query_map(query), seq)
# print(result)
resultss = blast.extend_hit(query, seq, (1, 16),3)
print(resultss)
results = blast.best_hit(query, seq, w)
print(results)

(0, 15, 7, 6)
(0, 0, 7, 6)


In [31]:
query = "AACTGA"
w = 3
sequences_obj = DB()
sequences_obj.extract_sequences()
for seq in sequences_obj.sequences:
    print(seq)



# print(sequences)
for seq in sequences:
    seq=''.join(i)
    print(seq)
    blast = Blast(w, seq)
    results = blast.best_hit(query, seq, w)
    print(results)
# (A,B,C,D)
# A = start position of the best match in the query
# B = starting position of the best match in the sequence
# C = best match size
# D = number of matches between nucleotides of the best match

TypeError: 'DB' object is not iterable

In [5]:
def search_db(query):
    cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG")
    cursor = cnx.cursor()
    cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
    sequences = cursor.fetchall()
    print(sequences)
    cursor.close()
    return sequences
search_db("AA")

[('TACCTTGTCCGAAAAGGAAAGACAAATCAAGAAGCAAACTGCTTTGGTTG',), ('CAACACGGGAAACCTCACCC',), ('GGGATGGAAGATGGCGGCCTA',), ('TCACATGGAGTACCACTAAGTTATTTGTAATATTTAATAAAGGAAATCTC',), ('GTTTGGTGAACTTCTGCCTA',), ('CCAAGAACAAATAACTTGTAAAATACACTGTTTTTTCACTGCAAATGTGT',), ('TGTGAATGAAAAAAGCAGCCAGTCAGTAGGAAACACTGTCTTGTATACAC',), ('TTTTTTTTTTTTTTTAACAATGGAGAAATCCATCAAAACATGGGTCATTT',), ('TTTTTTAAGTCAGTAAAAAGTAGGCACATCTTTATTTGGCAATGTCATAT',), ('GAATTCGTGCGCAACGCAATCCGGAAGTACAAGCAACTGCAGAAGCTGGT',), ('GGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC',), ('NNNNNNNAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTA',)]


[('TACCTTGTCCGAAAAGGAAAGACAAATCAAGAAGCAAACTGCTTTGGTTG',),
 ('CAACACGGGAAACCTCACCC',),
 ('GGGATGGAAGATGGCGGCCTA',),
 ('TCACATGGAGTACCACTAAGTTATTTGTAATATTTAATAAAGGAAATCTC',),
 ('GTTTGGTGAACTTCTGCCTA',),
 ('CCAAGAACAAATAACTTGTAAAATACACTGTTTTTTCACTGCAAATGTGT',),
 ('TGTGAATGAAAAAAGCAGCCAGTCAGTAGGAAACACTGTCTTGTATACAC',),
 ('TTTTTTTTTTTTTTTAACAATGGAGAAATCCATCAAAACATGGGTCATTT',),
 ('TTTTTTAAGTCAGTAAAAAGTAGGCACATCTTTATTTGGCAATGTCATAT',),
 ('GAATTCGTGCGCAACGCAATCCGGAAGTACAAGCAACTGCAGAAGCTGGT',),
 ('GGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC',),
 ('NNNNNNNAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTA',)]